In [2]:
from langchain.embeddings import OllamaEmbeddings, HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.llms import CTransformers, Ollama

In [3]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [4]:
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [5]:
#DATA INGESTION
def load_pdf(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)
    docs = loader.load()
    return docs

In [7]:
extracted_data = load_pdf(data="D:\My_Coding_Files\medical-chatbot-using-phi3mini\data")

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Arjo\AppData\Local\Temp\ipykernel_14816\1112340137.py:1: SyntaxWarning: invalid escape sequence '\M'
  extracted_data = load_pdf(data="D:\My_Coding_Files\medical-chatbot-using-phi3mini\data")


In [8]:
len(extracted_data)

637

In [10]:
#DATA TRANSFORMATION
#CREATING TEXT CHUNKS
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
docs = text_split(extracted_data=extracted_data)

In [12]:
len(docs)

7020

In [13]:
#Use embedding model
def hugging_face_embed_docs():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

def ollama_embed_docs():
    embeddings = OllamaEmbeddings(model="llama2")
    return embeddings

In [14]:
embeddings = ollama_embed_docs()
hugging_embeddings = hugging_face_embed_docs()

d:\My_Coding_Files\medical-chatbot-using-phi3mini\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
d:\My_Coding_Files\medical-chatbot-using-phi3mini\venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\My_Coding_Files\medical-chatbot-using-phi3mini\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Do

In [15]:
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='llama2', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [16]:
hugging_embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
vector_store = FAISS.from_documents(docs, hugging_embeddings)

In [18]:
vector_store

In [19]:
sample_ans = vector_store.similarity_search("What is an allergy", k=3)
sample_ans

[Document(page_content='When thisoccurs, an allergy develops against the offending sub-stance (an allergen.)', metadata={'source': 'D:\\My_Coding_Files\\medical-chatbot-using-phi3mini\\data\\Medical_book_1.pdf', 'page': 128}),
 Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-', metadata={'source': 'D:\\My_Coding_Files\\medical-chatbot-using-phi3mini\\data\\Medical_book_1.pdf', 'page': 135}),
 Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={'source': 'D:\\My_Coding_Files\\

In [20]:
prompt_template = """
    Use the following pieces of information to answer the user's question.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    If you know the answer, answer it in a crisp and descriptive way. Use around 5 to 6 sentences to explain.
    Context: {context}
    Question: {question}
    Only return the helpful answer below and nothing else.
    At the end of each answer which you know, don't end it with any statement thanking the user for asking you the question. Just end your answer normally without thanking the user. Just anser what is necessary. No thank you to the user is needed
    If the user tells "Thank you", or anything related to it, then reply in short, thanking the user to ask questions to you and telling him to consult you again in the future in case of any help.
    Assistant : 
    """

In [21]:
PROMPT = PromptTemplate(template = prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt" : PROMPT}

In [41]:
# llm = CTransformers(model = r"C:\Users\Arjo\.ollama\models\blobs\sha256-8934d96d3f08982e95922b2b7a2c626a1fe873d7c3b06e8e56d7bc0a1fef9246", model_type='llama2', config={'max_new_tokens' : 512, 'temperature' : 0.8})

In [22]:
llm = Ollama(model="phi3:mini", temperature=0.8, num_predict=1024)

In [23]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff',
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True,
    retriever = vector_store.as_retriever(search_kwargs = {'k' : 2})
)

In [24]:
while True:
    user_input = input(f"Input Prompt : ")
    result = qa({'query' : user_input})
    print("Response : ", result['result'])

d:\My_Coding_Files\medical-chatbot-using-phi3mini\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001EB33F3D010>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))